# Fitting or: how I learned to stop worrying and make that darng model pass through my data-points

> **(First question: do you get the reference?)**

We have obtained (either because we have measured it or because it has been given to us) a **SUMMARY STATISTICS** of our population of objects.

We are scientists and we want to represent summary statistics with analytical laws:

- because those can be made data-set independent and passed around more easily
- because a summary statistics measured on some population can be compared to that measured on some other population
- because they can give us physical insights

In [ ]:
import numpy, os
import matplotlib.pyplot as plt
basedir = '../datasets'

Let's load the dataset we have stored before

In [ ]:
infile = os.path.join(basedir, 'SFR_density_function.npz')
infile

This to check that the file exists

In [ ]:
!ls $infile

It's an ``npz`` file so I can load it directly with numpy using:

In [ ]:
data = numpy.load(infile) # remember this uses the pickle protocol by default

**WHY IS IT OK TO PASS AROUND STUFF WITH PICKLE PROTOCOL HERE?**

I can check what's is stored inside

In [ ]:
data.keys()

And I can unpack the values tuple

(as much as I would have done with a dictionary)

In [ ]:
lSFR, nSFR, nSFR_e = data.values()

And we can reproduce here the previous plot just for checking everything looks as expected:

In [ ]:
fig, ax = plt.subplots(1,1)
_ = ax.set(
    xscale='log', yscale='log',
    xlabel='$\\log [\\mathrm{SFR}/(M_\\star\\cdot\\mathrm{yr}^{-1})]$',
    ylabel='$\\log n[\\mathrm{SFR}/(M_\\star\\cdot\\mathrm{yr}^{-1}\\cdot Mpc^{-3})]$'
)
_ = ax.errorbar(10**lSFR, nSFR, yerr=nSFR_e, 
                marker='o', linestyle='none', color='k', label='data')

### Guess what? This is a Schechter function

$$n(x) = \mathcal{N} \left(\dfrac{x}{c}\right)^{(1-\alpha)} \exp\left(-\dfrac{x}{c}\right)$$

which it's basically a power-law with an exponential cut-off.
**Give me some examples of Schechter-like functions please.**

In [ ]:
def schechter_function ( lx, a = 1.5, n = 0.01, c = 1.0 ) :
    """Schechter-like function
    
    Parameters
    ----------
    x : scalar or sequence
        log10 values
    a : scalar
        alpha index of the power-low
    n : scalar
        normalization
    c : scalar
        critical value
        
    Returns
    -------
    : scalar or sequence
        schechter function computed per each grid point
        for given set of parameters
    """
    y = 10**(numpy.array( lx ) - c)
    return n * y**(1-a) * numpy.exp( - y )

This depends on **3 PARAMETERS**: ``a``, ``n``, ``c``

In [ ]:
fig, ax = plt.subplots(1,1)
_ = ax.set(
    xscale='log', yscale='log',
    xlabel='$\\log [\\mathrm{SFR}/(M_\\star\\cdot\\mathrm{yr}^{-1})]$',
    ylabel='$\\log n[\\mathrm{SFR}/(M_\\star\\cdot\\mathrm{yr}^{-1}\\cdot Mpc^{-3})]$'
)
_ = ax.plot(10**lSFR, schechter_function(lSFR), ls='--', color='gray', label='model')
_ = ax.errorbar(10**lSFR, nSFR, yerr=nSFR_e, 
                marker='o', linestyle='none', color='k', label='data')
_ = ax.legend()

### EXERCISE: How to quantify your confidence in the model?

For simplicity let's assume that the errors are Gaussian (which is a reasonable safe assumption)

(even though we have built them as Poissonian, but we are going to ignore this)

In [ ]:
# write here the estimator
def confidence ( data, error, model ) :
    data = numpy.asarray(data)
    error = numpy.asarray(error)
    model = numpy.asarray(model)
    return (
        (( data - model )/error)**2
    ).sum()

In [ ]:
confidence( nSFR, nSFR_e, schechter_function(lSFR) )/(nSFR.size-3)

### EXERCISE: Now what?

In [ ]:
help(schechter_function)

#### Grid methods

Compute a set of models on a grid in the parameter space.
Here the parameters are 3, therefore the parameter space i 3-dimensional.

In [ ]:
rng = numpy.random.default_rng(seed=555)

Instead of making an actual grid, with the risk that the number of elements blows up with the growth of the parameter space dimension, we are instead uniformly sampling a box, in the parameter space, assuming some limits on each parameter's dimension.

In [ ]:
pgrid = rng.uniform( 
    low=[0.,0.,-2], 
    high=[3, 3, 1], 
    size=(1024**2,3) 
)
pgrid.shape

With the command above I have generated a matrix, the first 4 lines look like this:

In [ ]:
pgrid[:4]

Where the first column corresponds to the first parameter (``a``) and so on.

I can pass these values to the ``schechter_function`` model we have defined above, modulo that I smartly organize them in a way that allows NumPy to **broadcast** the function across the whole $26\times1024^2$ space

i.e. ``pgrid.T`` transposes the original matrix, so that now it is a $3\times1024^2$ matrix, where the 3 lines contain $1024^2$ random samples each

In [ ]:
%%time
models_grid = schechter_function(
    lSFR, 
    pgrid.T[0][:,numpy.newaxis], 
    pgrid.T[1][:,numpy.newaxis], 
    pgrid.T[2][:,numpy.newaxis]
)

In [ ]:
models_grid.shape

The ``models`` matrix contains $1024^2$ estimate of the model, one per each random position in the parameter space.

To measure the $\chi^2$ of these models exploiting broadcasting, we have to slightly modify the function we have defined above:

In [ ]:
def confidence ( data, error, model ) :
    data = numpy.asarray(data)
    error = numpy.asarray(error)
    model = numpy.asarray(model)
    return (
        (( data - model )/error)**2
    ).sum( axis = -1 ) # <----------- Here

The argument I have passed to the ``sum`` function, i.e. ``axis=-1``, tells the function to perfom the sum of the elements only on the last axis.

- if the above function is called on a single model, nothing changes from the behaviour of before, the sum is done for the 26 data-points
- if instead the function is called on our $1024^2\times26$ grid, it is done only on the last dimension, providing us with $1024^2$ values of $\chi^2$

In [ ]:
chi2_grid = confidence( nSFR, nSFR_e, models_grid )

In [ ]:
chi2_grid.shape

We can assign the index of the smallest $\chi^2$ value found to some variable.

In [ ]:
ibest_grid = chi2_grid.argmin()

the above index tells me that ``models[ibest]`` is the best-fitting model of my sample, with a reduced $\chi^2$ value of

In [ ]:
chi2_grid.min()/(nSFR.size-3)

So let's plot the new model and also compare the standardised residuals

In [ ]:
fig, axs = plt.subplots(2,1, sharex=True, gridspec_kw={'height_ratios':(2,1), 'hspace':0.0})

# first sub-plot
_ = axs[0].set(
    xscale='log', yscale='log',
    ylabel='$\\log n[\\mathrm{SFR}/(M_\\star\\cdot\\mathrm{yr}^{-1}\\cdot Mpc^{-3})]$'
)
_ = axs[0].plot(10**lSFR, schechter_function(lSFR), ls='--', color='gray', label='model naive')
_ = axs[0].plot(10**lSFR, models_grid[ibest_grid], ls='-', color='orange', label='model grid')
_ = axs[0].errorbar(10**lSFR, nSFR, yerr=nSFR_e, 
                marker='o', linestyle='none', color='k', label='data')
_ = axs[0].legend()

# second sub-plot
_ = axs[1].set(
    xscale='log',
    xlabel='$\\log [\\mathrm{SFR}/(M_\\star\\cdot\\mathrm{yr}^{-1})]$',    
    ylabel='$\chi$'
)
_ = axs[1].axhline(0.0, color='orange', ls='-')
_ = axs[1].plot(10**lSFR, (nSFR - models_grid[ibest_grid])/nSFR_e, marker='o', color='k', ls='none')

Standardised residuals are defined as

$$\chi_i = \dfrac{\text{data} - \text{model}}{\text{error}}$$

#### SciPy's minimization algorithms
[``scipy.optimize.minimize``](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)

Can probably be classified as a grid algorithm? I don't know actually.

Anyways, these have implemented smarter algorithms w.r.t. what we did above..

When to use them?

- some people's answer is "always" (but we do not like this answer)
- **the parameter space is small** (i.e. the **size of the prior** is small)
- when we have an unknown dataset and we want some quick test to perform, modulo that:
    * the dataset is small
    * the parameter space is small
- when we don't know what are reasonable limits for our grid (some of these minimization algorithms can be run without constraining the parameter space but using an initial guess instead)

> **NOTE** Those listed above are **rules of thumb**, there are exceptions that you will learn to spot with experience.

> **NOTE ALSO** The above arguments also apply to the naive grid method we have seen before

Also, there are very sofisticated algorithms implemented in ``minimize``, such as algorithms that exploit Hessians and Jacobians of the objective function. Therefore in some cases (when such functions can be computed easily, these methods could be quite helpful).


> A small digression on *Bayesianity* of your statistical analysis:
>
> **when is it possible to say that what we did is Bayesian?**

In [ ]:
from scipy.optimize import minimize

In [ ]:
help(minimize)

We need an *objective function*, what does it mean?

It means that we need a function, depending on the free parameters of the model, that has to be minimized.

- the schechter function is the one that depends on the parameters, but is not what we need to minimize
- the $\chi^2$ is the function we need to minimize, but with our definition above it does not depend directly on the parameters

so we just define a new function which is the combination of the ones above:

In [ ]:
def objective_function ( p, x, y, e ) :
    return confidence(
        y, e, 
        schechter_function( x, *p )
    )

Than ``minimize`` needs a starting point, we are going to use the default values of the parameters of ``schechter_function``

In [ ]:
p0 = [1.5, 0.01, 1.0]

And that's it, we are ready to run minimize, note though that ``objective_function`` requires some additional arguments besides the free parameters, we can pass them as a tuple of ``args``

In [ ]:
%time res = minimize( objective_function, [1.5, 0.01, 1.0], args = (lSFR, nSFR, nSFR_e) )

In [ ]:
res

We can check also in this case the reduced $\chi^2$ value

In [ ]:
confidence( nSFR, nSFR_e, schechter_function(lSFR, *res.x)) / (nSFR.size-3)

For reference, with the grid method we got

In [ ]:
chi2_grid[ibest_grid]/ (nSFR.size-3)

And we can plot the result

In [ ]:
fig, axs = plt.subplots(2,1, sharex=True, gridspec_kw={'height_ratios':(2,1), 'hspace':0.0})

# first sub-plot
_ = axs[0].set(
    xscale='log', yscale='log',
    ylabel='$\\log n[\\mathrm{SFR}/(M_\\star\\cdot\\mathrm{yr}^{-1}\\cdot Mpc^{-3})]$'
)
_ = axs[0].plot(10**lSFR, schechter_function(lSFR), ls='--', color='gray', label='model naive')
_ = axs[0].plot(10**lSFR, models_grid[ibest_grid], ls='-', color='orange', label='model grid')
_ = axs[0].plot(10**lSFR, schechter_function(lSFR, *res.x), ls='-', color='green', label='model scipy')
_ = axs[0].errorbar(10**lSFR, nSFR, yerr=nSFR_e, 
                marker='o', linestyle='none', color='k', label='data')
_ = axs[0].legend()

# second sub-plot
_ = axs[1].set(
    xscale='log',
    xlabel='$\\log [\\mathrm{SFR}/(M_\\star\\cdot\\mathrm{yr}^{-1})]$',    
    ylabel='$\chi$'
)
_ = axs[1].axhline(0.0, ls='-', color='green')
_ = axs[1].plot(10**lSFR, (nSFR - schechter_function(lSFR, *res.x))/nSFR_e, marker='o', color='k', ls='none')

#### Monte-Carlo Markov Chain (MCMC) methods

We will use [``emcee``](https://emcee.readthedocs.io/en/stable/), but I will give an explanation of what a markov-chain **AT THE BLACKBOARD**

First of all, instead of minimizing the $\chi^2$ we will **MAXIMIZE THE POSTERIOR PROBABILITY**.

**QUESTION:** Therefore, what are we going to work with?

In [ ]:
def log_likelihood ( p, x, y, e ) :
    
    a, ln, c = p
    n = 10.**ln
    m = schechter_function( x, a, ln, c )
    chi2 = confidence( y, e, m )
    
    return - 0.5 * chi2

It is good practice to always test a function when we define it:

In [ ]:
log_likelihood( p0, lSFR, nSFR, nSFR_e )

In [ ]:
def log_prior ( p, lims ) :
    
    p = numpy.asarray(p)
    lims = numpy.asarray(lims)
    
    if numpy.all( (lims[0] < p)&(p < lims[1]) ) :
        return 0.0
    return -numpy.inf

In [ ]:
lims = numpy.array([[1.,-1.,-1], 
                    [2., +1, 2]])

In [ ]:
log_prior( p0, lims )

In [ ]:
def log_prob ( p, x, y, e, lims ) :
    
    # compute the prior
    lp = log_prior( p, lims )
    if not numpy.isfinite( lp ) :
        return -numpy.inf
    
    # return the log( likelihood * prior ) = log(likelihood) + log(prior)
    return log_likelihood( p, x, y, e ) + lp

In [ ]:
log_prob( p0, lSFR, nSFR, nSFR_e, lims ) 

We are all set up to run our first MCMC sampling

In [ ]:
ndim, nwalkers = 3, 16
pstart = rng.uniform( 
    low =lims[0], 
    high=lims[1], 
    size=(nwalkers, ndim),
)

In [ ]:
pstart.shape

In [ ]:
import emcee

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, args=(lSFR, nSFR, nSFR_e, lims))

In [ ]:
state = sampler.run_mcmc(pstart, 8*1024, progress = True)

How do we extract information on this run now?

It is an MCMC therefore samples are drawn, then accepted or discarded. We can have a first diagnostic by checking what was the acceptance fraction of this run:

In [ ]:
sampler.acceptance_fraction.mean()

typically, values $\gtrsim 0.3\div0.4 \equiv 30\div40\%$ are symptom of a good run. So everything's fine up to here.

> Note that all the values returned by the ``sampler`` object are arrays or matrices with one of the dimension equal to the number of walkers, i.e. all walkers are treated separately

In [ ]:
samples = sampler.get_chain(flat=True) # <--- flat=True means we are flattening on the walkers dimension

In [ ]:
samples

In [ ]:
samples.shape

Each element of this ``samples`` matrix is a position in the parameter space, for which an estimate of the posterior probability has been computed:

In [ ]:
loglike = sampler.get_log_prob(flat=True)

We can retrieve the best-fitting value by finding where ``logprob`` (i.e. the **log-likelihood**) was maximum:

In [ ]:
ibest_mcmc = loglike.argmax()

We have to find, among the samples, the position in the parameter space where the maximum a-posteriori probability has been found:

In [ ]:
pbest = samples[ibest_mcmc]

In [ ]:
pbest

And that's it! We got our best-fitting parameters, we can therefore, and once again, compute the $\chi^2$ and plot the resulting best-fitting model

In [ ]:
confidence(nSFR, nSFR_e, schechter_function(lSFR, *pbest))/(nSFR.size-3)

Which is similar to the value found with the SciPy's algorithm.

In [ ]:
fig, axs = plt.subplots(2,1, sharex=True, gridspec_kw={'height_ratios':(2,1), 'hspace':0.0})

# first sub-plot
_ = axs[0].set(
    xscale='log', yscale='log',
    ylabel='$\\log n[\\mathrm{SFR}/(M_\\star\\cdot\\mathrm{yr}^{-1}\\cdot Mpc^{-3})]$'
)
_ = axs[0].plot(10**lSFR, schechter_function(lSFR), ls='--', color='gray', label='model naive')
_ = axs[0].plot(10**lSFR, models_grid[ibest_grid], ls='-', color='orange', label='model grid')
_ = axs[0].plot(10**lSFR, schechter_function(lSFR, *res.x), ls='-', color='green', label='model scipy')
_ = axs[0].plot(10**lSFR, schechter_function(lSFR, *pbest), ls='-', color='red', label='model emcee')
_ = axs[0].errorbar(10**lSFR, nSFR, yerr=nSFR_e, 
                marker='o', linestyle='none', color='k', label='data')
_ = axs[0].legend()

# second sub-plot
_ = axs[1].set(
    xscale='log',
    xlabel='$\\log [\\mathrm{SFR}/(M_\\star\\cdot\\mathrm{yr}^{-1})]$',    
    ylabel='$\chi$'
)
_ = axs[1].axhline(0.0, ls='-', color='red')
_ = axs[1].plot(10**lSFR, (nSFR - schechter_function(lSFR, *pbest))/nSFR_e, marker='o', color='k', ls='none')

How do I check whether my MCMC run has converged?
We can first check how the likelihood has varied in time:

In [ ]:
plt.plot(loglike[1024::128])

This is already a good symptom, it is telling us that, after some steps, the value of the likelihood converged and did not change much.

> Note that I have sub-sampled the array and cut out some initial steps (which are called **BURN-IN**)

There are further diagnostics to evaluate your run, these are listed in the [``emcee`` documentation](https://emcee.readthedocs.io/en/stable/tutorials/autocorr/) and usually they depend on the sampling algorithm (and library) you are using.

Last but not least, since we have run a parameter space sampling, we can see the posterior probability distributed on the parameter space. Doing this plot starting from ``matplotlib`` functions is possible but hard, nobody does it, people uses instead libraries designed for that.

One of these libraries is [``¢orner``](https://corner.readthedocs.io/en/latest/)

We are using instead [GetDist](https://getdist.readthedocs.io/en/latest/index.html) because I am more familiar with it and it has a nice [plot gallery](https://getdist.readthedocs.io/en/latest/plot_gallery.html) from where to fish the plot that better suits your needs.

You can choose your favourite one.

In [ ]:
from getdist import plots, MCSamples

In [ ]:
mcsamples = MCSamples(
    samples = samples,
    loglikes = -loglike,
    names = ['a', 'logN', 'c'], 
    sampler = 'mcmc',
    settings = { 
        'mult_bias_correction_order' : 1,
        'smooth_scale_2D' : .5, 
        'smooth_scale_1D' : .5, 
        'fine_bins': 512,
        'fine_bins_2D' : 512,
    }
)

In [ ]:
g = plots.get_subplot_plotter()
g.triangle_plot(
    [mcsamples], 
    filled=True, title_limit=1
)